# RAG


In [142]:
!pip install -qU langchain langchain-openai langchain_community langchain_experimental langchain-text-splitters tiktoken pypdf faiss-cpu wikipedia langchainhub

In [139]:
import os

os.environ["OPENAI_API_KEY"] = 'sk-*****************************************************'

## RAG QA Bot Example

In [133]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

**WebBaseLoader + bs4**

In [134]:
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/437/0000378416",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body",
                             "media_end_head_title"]},
        )
    ),
)

In [135]:
docs = loader.load()
print(f"len of docs: {len(docs)}")
docs

len of docs: 1


[Document(page_content="\n출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책\n\n\n[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다.2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다.[오현석/부영그룹 직원 : 아이 키우는 데 금전적으로 많이 힘든 세상이잖아요. 교육이나 생활하는 데 큰 도움이 될 거라 생각합니다.]만약 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 뜻도 밝혔습니다.[이중근/부영그룹 회장 : 3년 이내에 세 아이를 갖는 분이 나올 것이고 따라서 주택을 제공할 수 있는 계기가 될 것으로 생각하고.][조용현/부영그룹 직원 : 와이프가 셋째도 갖고 싶어 했는데 경제적 부담 때문에 부정적이었거든요. (이제) 긍정적으로 생각할 수 있을 것 같습니다.]오늘 행사에서는, 회사가 제공하는 출산장려금은 받는 직원들의 세금 부담을 고려해 정부가 면세해달라는 제안도 나왔습니다.이같은 출산장려책은 점점 확산하는 분위기입니다.법정기간보다 육아

**PDF**

**CSV**

**Text**

**Text splliting**

In [136]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents(docs)
len(splits)

3

**vectorization**

In [140]:
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

**Prompt**

In [143]:
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

**create Chain**

In [144]:
from langchain.callbacks.base import BaseCallbackHandler


class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs):
        print(token, end="", flush=True)


llm = ChatOpenAI(
    model_name="gpt-4-turbo-preview",
    temperature=0,
    streaming=True,
    callbacks=[StreamCallback()],
)


def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


# 체인을 생성합니다.
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

**Execution**

In [145]:
rag_chain.invoke("부영그룹의 출산 장려 정책에 대해 설명해주세요.")

부영그룹은 출산 장려 정책으로 2021년 이후 태어난 직원 자녀에게 1억원씩 지원하며, 연년생이나 쌍둥이 자녀가 있을 경우 총 2억원을 지급합니다. 또한, 셋째 자녀를 낳는 경우 국민주택을 제공하겠다는 계획도 밝혔습니다. 이러한 정책은 직원들에게 금전적인 지원과 함께 경제적 부담을 줄여주어 출산을 장려하기 위한 것입니다.

'부영그룹은 출산 장려 정책으로 2021년 이후 태어난 직원 자녀에게 1억원씩 지원하며, 연년생이나 쌍둥이 자녀가 있을 경우 총 2억원을 지급합니다. 또한, 셋째 자녀를 낳는 경우 국민주택을 제공하겠다는 계획도 밝혔습니다. 이러한 정책은 직원들에게 금전적인 지원과 함께 경제적 부담을 줄여주어 출산을 장려하기 위한 것입니다.'

In [146]:
rag_chain.invoke("부영그룹은 출산 직원에게 얼마의 지원을 제공하나요?")

부영그룹은 출산 직원에게 1억원을 지원합니다. 2021년 이후 태어난 직원 자녀에게 이 지원금을 제공하며, 연년생이나 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다. 셋째 아이를 낳는 경우에는 국민주택을 제공하겠다는 계획도 있습니다.

'부영그룹은 출산 직원에게 1억원을 지원합니다. 2021년 이후 태어난 직원 자녀에게 이 지원금을 제공하며, 연년생이나 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다. 셋째 아이를 낳는 경우에는 국민주택을 제공하겠다는 계획도 있습니다.'

In [147]:
print(rag_chain.invoke("정부의 저출생 대책을 bullet points 형식으로 작성해 주세요."))

- 매달 부모 급여 지급: 0세 아이는 100만원으로 인상
- 첫만남이용권 및 아동수당 포함하여 아이 돌까지 1년 동안 총 1520만원 지원
- 지자체별 추가 지원: 예를 들어, 인천시는 새로 태어난 아기에게 18살 될 때까지 1억원 지급, 광주시는 17살 될 때까지 7400만원 지급- 매달 부모 급여 지급: 0세 아이는 100만원으로 인상
- 첫만남이용권 및 아동수당 포함하여 아이 돌까지 1년 동안 총 1520만원 지원
- 지자체별 추가 지원: 예를 들어, 인천시는 새로 태어난 아기에게 18살 될 때까지 1억원 지급, 광주시는 17살 될 때까지 7400만원 지급


**Returning Source**

In [154]:
from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [156]:
rag_chain_with_source.invoke("부영그룹은 출산 직원에게 얼마의 지원을 제공하나요?")

부영그룹은 출산 직원에게 1억원을 지원합니다. 2021년 이후 태어난 직원 자녀에게 이 지원금을 제공하며, 연년생이나 쌍둥이 자녀가 있을 경우 총 2억원을 받게 됩니다. 셋째 아이를 낳는 경우에는 국민주택을 제공하겠다는 계획도 밝혔습니다.

{'context': [Document(page_content="[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다.2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다.[오현석/부영그룹 직원 : 아이 키우는 데 금전적으로 많이 힘든 세상이잖아요. 교육이나 생활하는 데 큰 도움이 될 거라 생각합니다.]만약 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 뜻도 밝혔습니다.[이중근/부영그룹 회장 : 3년 이내에 세 아이를 갖는 분이 나올 것이고 따라서 주택을 제공할 수 있는 계기가 될 것으로 생각하고.][조용현/부영그룹 직원 : 와이프가 셋째도 갖고 싶어 했는데 경제적 부담 때문에 부정적이었거든요. (이제) 긍정적으로 생각할 수 있을 것 같습니다.]오늘 행사에서는, 회사가 제공하는 출산장려금은 받는 직원들의 세금 부담을 고려해 정부가 면세해달라는 제안도 나왔습니다.이같은 출산장려책은 점점 확산하는 분위기입니다.법정기간보다 육아휴직을 길게 주거나, 남성 직원의 육아휴직을 의무

**Add chat history**

In [157]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """채팅 기록과 최근 사용자 질문이 주어졌습니다.\
채팅 기록의 컨텍스트를 참조할 수 있는 독립형 질문을 \
채팅 기록 없이도 이해할 수 있습니다. 질문에 대답하지 마십시오. \
필요한 경우 다시 구성하고, 그렇지 않은 경우에는 그대로 반환하세요."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()

In [158]:
qa_system_prompt = """당신은 질의응답 업무의 보조자입니다.\
다음 검색된 컨텍스트 조각을 사용하여 질문에 답하세요. \
답을 모르면 모른다고 하면 됩니다. \
최대 3개의 문장을 사용하고 대답은 간결하게 하세요.\

{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)


def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["question"]


rag_chain = (
    RunnablePassthrough.assign(
        context=contextualized_question | retriever | format_docs
    )
    | qa_prompt
    | llm
)

In [160]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "부영그룹은 출산 직원에게 얼마의 지원을 제공하나요?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg])

부영그룹은 출산 직원에게 1억원의 지원을 제공합니다. 해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다. 만약 셋째까지 낳는 경우에는 국민주택을 제공하겠다는 뜻도 밝혔습니다.

In [161]:
second_question = "부영그룹의 출산 장려 정책에 대해 설명해주세요."
ai_msg = rag_chain.invoke({"question": second_question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=second_question), ai_msg])

부영그룹의 출산 장려 정책에 대해 설명해주세요.부영그룹은 파격적인 저출생 정책의 일환으로, 2021년 이후 태어난 직원 자녀에게 1억원씩 지원하며, 총 70억원을 지원할 계획입니다. 연년생이나 쌍둥이 자녀가 있는 경우 총 2억원을 받게 되며, 셋째 자녀를 낳는 경우에는 국민주택을 제공하겠다고 밝혔습니다. 이 정책은 금전적 부담을 덜어주고 출산을 장려하기 위한 목적으로, 회사는 이 정책을 지속적으로 이어가기로 했습니다.

In [162]:
chat_history

[HumanMessage(content='부영그룹은 출산 직원에게 얼마의 지원을 제공하나요?'),
 AIMessage(content='부영그룹은 출산 직원에게 1억원의 지원을 제공합니다. 해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다. 만약 셋째까지 낳는 경우에는 국민주택을 제공하겠다는 뜻도 밝혔습니다.', response_metadata={'finish_reason': 'stop'}),
 HumanMessage(content='부영그룹의 출산 장려 정책에 대해 설명해주세요.'),
 AIMessage(content='부영그룹은 파격적인 저출생 정책의 일환으로, 2021년 이후 태어난 직원 자녀에게 1억원씩 지원하며, 총 70억원을 지원할 계획입니다. 연년생이나 쌍둥이 자녀가 있는 경우 총 2억원을 받게 되며, 셋째 자녀를 낳는 경우에는 국민주택을 제공하겠다고 밝혔습니다. 이 정책은 금전적 부담을 덜어주고 출산을 장려하기 위한 목적으로, 회사는 이 정책을 지속적으로 이어가기로 했습니다.', response_metadata={'finish_reason': 'stop'})]